This notebook shows how to fine-tune a BERT model (from huggingface) for our dataset recognition task.

Note that internet is needed during the training phase (for downloading the bert-base-cased model). Internet can be turned off during prediction.

## Install packages

In [ ]:
!pip install datasets --no-index --find-links=file:///kaggle/input/coleridge-packages/packages/datasets
!pip install ../input/coleridge-packages/seqeval-1.2.2-py3-none-any.whl
!pip install ../input/coleridge-packages/tokenizers-0.10.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/coleridge-packages/transformers-4.5.0.dev0-py3-none-any.whl

# Import

In [ ]:
import os
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

random.seed(123)
np.random.seed(456)

In [ ]:
# copy my_seqeval.py to the working directory because the input directory is non-writable
!cp /kaggle/input/coleridge-packages/my_seqeval.py ./

# Hyper-parameters

In [ ]:
MAX_LENGTH = 48 # max no. words for each sentence.
OVERLAP = 16 # if a sentence exceeds MAX_LENGTH, we split it to multiple sentences with overlapping

MAX_SAMPLE = None # set a small number for experimentation, set None for production.

# Load data

In [ ]:
train_path = '../input/coleridgeinitiative-show-us-the-data/train.csv'
paper_train_folder = '../input/coleridgeinitiative-show-us-the-data/train'

train = pd.read_csv(train_path)
train = train[:MAX_SAMPLE]
print(f'No. raw training rows: {len(train)}')

Group by publication, training labels should have the same form as expected output.

In [ ]:
train = train.groupby('Id').agg({
    'pub_title': 'first',
    'dataset_title': '|'.join,
    'dataset_label': '|'.join,
    'cleaned_label': '|'.join
}).reset_index()

print(f'No. grouped training rows: {len(train)}')

In [ ]:
papers = {}
for paper_id in train['Id'].unique():
    with open(f'{paper_train_folder}/{paper_id}.json', 'r') as f:
        paper = json.load(f)
        papers[paper_id] = paper

# Filtering validation labels from training data

In [ ]:
validation_labels = ['adni', "alzheimer's disease neuroimaging initiative (adni)",
                     'alzheimers disease neuroimaging initiative', 'alzheimer s disease neuroimaging initiative adni ',
                     "ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI)"
#                     'rural-urban continuum codes', 'rural urban continuum codes',
#                     'baccalaureate and beyond', 'baccalaureate and beyond longitudinal study',
                    ]

# Actual filtering is done below

# Transform data to NER format

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

def totally_clean_text(txt):
    txt = clean_text(txt)
    txt = re.sub(' +', ' ', txt)
    return txt

def clean_training_text(txt):
    """
    similar to the default clean_text function but without lowercasing.
    """
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt)).strip()

def shorten_sentences(sentences):
    short_sentences = []
    for sentence in sentences:
        words = sentence.split()
        if len(words) > MAX_LENGTH:
            for p in range(0, len(words), MAX_LENGTH - OVERLAP):
                short_sentences.append(' '.join(words[p:p+MAX_LENGTH]))
        else:
            short_sentences.append(sentence)
    return short_sentences

def find_sublist(big_list, small_list):
    all_positions = []
    for i in range(len(big_list) - len(small_list) + 1):
        if small_list == big_list[i:i+len(small_list)]:
            all_positions.append(i)
    
    return all_positions

def tag_sentence(sentence, labels): # requirement: both sentence and labels are already cleaned
    sentence_words = sentence.split()
    
    if labels is not None and any(re.findall(f'\\b{label}\\b', sentence)
                                  for label in labels): # positive sample
        nes = ['O'] * len(sentence_words)
        for label in labels:
            label_words = label.split()

            all_pos = find_sublist(sentence_words, label_words)
            for pos in all_pos:
                nes[pos] = 'B'
                for i in range(pos+1, pos+len(label_words)):
                    nes[i] = 'I'

        return True, list(zip(sentence_words, nes))
        
    else: # negative sample
        nes = ['O'] * len(sentence_words)
        return False, list(zip(sentence_words, nes))

In [ ]:
cnt_pos, cnt_neg = 0, 0 # number of sentences that contain/not contain labels
ner_data = []
count_valid = 0

pbar = tqdm(total=len(train))
for i, id, dataset_label, cleaned_label in train[['Id', 'dataset_label', 'cleaned_label']].itertuples():
    
    # Initial filter of adni labels
    if not "adni" in dataset_label.lower() and not "disease neuroimaging" in dataset_label.lower() and not "disease neuroimaging" in cleaned_label.lower() and not "adni" in cleaned_label.lower():
        # paper
        paper = papers[id]
        valid_occurrence = False
        
        text_1 = '. '.join(section['text'] for section in paper).lower()
        text_2 = totally_clean_text(text_1)
        # Second filter of adni occurrences in actual text
        for valid_label in validation_labels:
            if valid_label in text_1 or valid_label in text_2:
                valid_occurrence = True
                count_valid += 1
                break
        
        if not valid_occurrence:
            # labels
            labels = dataset_label.split('|')
            labels = [clean_training_text(label) for label in labels]

            # sentences
            sentences = set([clean_training_text(sentence) for section in paper 
                         for sentence in section['text'].split('.') 
                        ])
            sentences = shorten_sentences(sentences) # make sentences short
            sentences = [sentence for sentence in sentences if len(sentence) > 10] # only accept sentences with length > 10 chars

            # positive sample
            for sentence in sentences:                    
                is_positive, tags = tag_sentence(sentence, labels)
                if is_positive:
                    cnt_pos += 1
                    ner_data.append(tags)
                elif any(word in sentence.lower() for word in ['data', 'study']): 
                    ner_data.append(tags)
                    cnt_neg += 1

            # process bar
            pbar.update(1)
            pbar.set_description(f"Training data size: {cnt_pos} positives + {cnt_neg} negatives")
    else:
        count_valid += 1
        
print(len(train))
print(count_valid)
# shuffling
random.shuffle(ner_data) # 'Alzheimers Disease Neuroimaging Initiative', 'ADNI', 'Alzheimer s Disease Neuroimaging Initiative ADNI'

write data to file.

In [ ]:
with open('train_ner.json', 'w') as f:
    for row in ner_data:
        words, nes = list(zip(*row))
        row_json = {'tokens' : words, 'tags' : nes}
        json.dump(row_json, f)
        f.write('\n')

# Fine-tune a BERT model for NER

In [ ]:
!python ../input/kaggle-ner-utils/kaggle_run_ner.py \
--model_name_or_path "allenai/scibert_scivocab_cased" \
--train_file './train_ner.json' \
--validation_file './train_ner.json' \
--num_train_epochs 1 \
--per_device_train_batch_size 16 \
--per_device_eval_batch_size 16 \
--save_steps 15000 \
--output_dir './output' \
--report_to 'none' \
--seed 123 \
--do_train 

In [ ]:
# Disregard below: output can be saved with quick save through advanced settings in the
# saving popup screen

# Saving locally to manually make Kaggle dataset (only in case save&commit doesn't work 
# because of GPU memory overload)
# !zip -r output.zip "./output"
# from IPython.display import FileLink
# FileLink(r'output.zip')

After the tuning finishes, we should find our model in './output'.